In [1]:
from typing import Dict

def format_entities_for_prompt(entities: Dict[str, str]) -> str:
    """
    Converts a dictionary of entities into a formatted string representation.
    
    Args:
        entities: Dictionary containing entity key-value pairs
        
    Returns:
        Formatted string with entities listed in bullet points
    """
    formatted_str = ""
    for key, value in entities.items():
        # Convert snake_case to Title Case
        formatted_key = " ".join(word.capitalize() for word in key.split('_'))
        formatted_str += f"        - {formatted_key}: '{value}'\n"
    return formatted_str.rstrip()


def get_user_prompt(message: str) -> str:
    
    
    user_prompt = f"""
    User's message: "{message}"
    
    # General conversation guidelines:
    - Please detect the pattern of user's query if it does not contain any removal request such "no user please", "delete product detail please" so on. Otherwise, default as a empty list {{"params2delete": []}}.
    - Please help me identify which parameters you want to remove from your query. This is all of the parameters: "from_date", "to_date", "product", "product_detail", "level", "user", "N/A"
    - Return your answer in JSON format with a single key "params2delete"

    # ***Example Scenarios:***
    
    - ***User***: "No username please"
    - ***Assistant***: {{"params2delete": ["username"]}}
    
    - ***User***: "I want to winlost report, no username and level please"
    - ***Assistant***: {{"params2delete": ["username", "level"]}}

    - ***User***: "Remove the date range"
    - ***Assistant***: {{"params2delete": ["from_date", "to_date"]}}

    - ***User***: "Don't include product"
    - ***Assistant***: {{"params2delete": ["product"]}}
    
    - ***User***: "I want to delete top"
    - ***Assistant***: {{"params2delete": ["top"]}}
    
    - ***User***: "Please default from_date"
    - ***Assistant***: {{"params2delete": ["from_date"]}}
    
    - ***User***: "I want to get winlost report for Sportsbook only"
    - ***Assistant***: {{"params2delete": []}}
    
    - ***User***: "Please roll back the product to default"
    - ***Assistant***: {{"params2delete": ["product"]}}
    """
    return user_prompt

In [2]:
import requests
import json

def delete_param_request(query: str) -> str:
    # API endpoint
    url = "https://ollama.selab.edu.vn/api/chat"

    # Construct the prompt for function determination with examples
    user_prompt = get_user_prompt(query)
    system_prompt = "You are a helpful assistant that identifies which parameters you want to remove from user's query."
    
    format_schema =  {
        "type": "object",
        "properties": {
            "params2delete": {
                "type": "array",
                "items": {
                    "type": "string",
                    "enum": [
                        "from_date",
                        "to_date",
                        "product",  
                        "product_detail",
                        "level",
                        "user",
                        "top"
                    ]
                }
            }
        },
        "required": ["params2delete"]
    }

    model = 'qwen2.5:14b'
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    headers = {"Content-Type": "application/json"}

    data = {
        "model": model,
        "messages": messages,
        "format": format_schema,
        "stream": False
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    return json.loads(response.json()['message']['content'])

In [19]:
delete_param_request("I want to get winlost report for Sportsbook only, no username and level please")

{'params2delete': ['user', 'level']}

In [20]:
delete_param_request("I want to delete that date range")

{'params2delete': ['from_date', 'to_date']}

In [21]:
delete_param_request("Please roll back the product to default")

{'params2delete': ['product']}

In [22]:
delete_param_request("I want to get turnover report please")

{'params2delete': []}

In [3]:
delete_param_request("i want winlost report in Number Game only")

{'params2delete': ['level', 'user', 'top']}

In [4]:
delete_param_request("No username please")

{'params2delete': ['user']}

In [5]:
delete_param_request("Delete top rank")

{'params2delete': ['top']}

In [6]:
delete_param_request("Please remove the product detail")

{'params2delete': ['product_detail']}

In [7]:
delete_param_request("No level please")

{'params2delete': ['level']}

In [8]:
delete_param_request("Destroy username please, I hate it")

{'params2delete': ['user']}

In [9]:
delete_param_request("Wrong product")

{'params2delete': ['product']}